In [1]:
import numpy as np
import cv2
import time
import imutils


def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

video_path = '.\\videos\\video2.mov'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 600
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))
    # start time to calculat FPS
    start = time.time()
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray
    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    print(f"{fps:.2f} FPS")
    cv2.imshow('flow', draw_flow(gray, flow))
    cv2.imshow('flow HSV', draw_hsv(flow))
    key = cv2.waitKey(5)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

16.95 FPS
17.76 FPS
17.81 FPS
19.81 FPS
15.85 FPS
17.55 FPS
18.62 FPS
18.28 FPS
16.81 FPS
18.97 FPS
16.41 FPS
19.30 FPS
20.68 FPS
20.98 FPS
22.39 FPS
24.22 FPS
23.49 FPS
22.74 FPS
24.52 FPS
22.90 FPS
21.37 FPS
19.84 FPS
24.40 FPS
21.95 FPS
19.68 FPS
20.02 FPS
20.64 FPS
22.29 FPS
20.87 FPS
22.09 FPS
21.21 FPS
21.69 FPS
20.28 FPS
22.53 FPS
23.44 FPS
21.47 FPS
24.76 FPS
24.47 FPS
24.36 FPS
22.59 FPS
23.21 FPS
23.73 FPS
23.49 FPS
21.59 FPS
23.26 FPS
24.29 FPS
24.28 FPS
23.55 FPS
22.66 FPS
16.95 FPS
15.87 FPS
16.15 FPS
23.68 FPS
25.52 FPS
21.86 FPS
22.52 FPS
22.30 FPS
22.29 FPS
22.69 FPS
22.92 FPS
22.86 FPS
23.56 FPS
23.20 FPS
19.74 FPS
23.52 FPS
24.56 FPS
23.66 FPS
22.67 FPS
24.14 FPS
23.95 FPS
22.68 FPS
15.30 FPS
16.55 FPS
21.00 FPS
22.98 FPS
24.03 FPS
23.03 FPS
22.97 FPS
19.55 FPS
21.17 FPS
21.70 FPS
21.59 FPS
20.87 FPS
21.58 FPS
20.08 FPS
19.62 FPS
19.27 FPS
21.01 FPS
23.86 FPS
21.31 FPS
24.22 FPS
22.85 FPS
24.66 FPS
23.02 FPS
23.74 FPS
23.47 FPS
23.33 FPS
22.83 FPS
23.76 FPS
24.20 FPS


In [2]:
import cv2
import numpy as np
import time
import imutils

def draw_flow(img, p0, p1, mask=None, color=(0, 255, 0), line_thickness=2):
    for i, (new, old) in enumerate(zip(p1, p0)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color, line_thickness)
        img = cv2.circle(img, (int(a), int(b)), 5, color, -1)
    return cv2.add(img, mask)

video_path = '.\\videos\\video2.mp4'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 600
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))


feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
p0 = cv2.goodFeaturesToTrack(prevgray, mask=None, **feature_params)

mask = np.zeros_like(prev)

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))

    start = time.time()
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    end = time.time()

    if end != start:
        fps = 1 / (end - start)
        print(f"{fps:.2f} FPS")
    else:
        print("Unable to calculate FPS (division by zero)")

    img = draw_flow(img, p0, p1, mask=mask)

    cv2.imshow('flow', img)
    p0 = p1.reshape(-1, 1, 2)

    key = cv2.waitKey(5)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


387.00 FPS
992.03 FPS
999.36 FPS
1000.31 FPS
Unable to calculate FPS (division by zero)
981.12 FPS
826.46 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
881.90 FPS
Unable to calculate FPS (division by zero)
703.03 FPS
Unable to calculate FPS (division by zero)
663.45 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
1000.07 FPS
799.68 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
915.59 FPS
996.98 FPS
999.60 FPS
1000.79 FPS
686.35 FPS
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
Unable to calculate FPS (division by zero)
996.75 FPS
Unable to calculate FPS (division by zero)
996.98 FPS
996.75 FPS
1000.79 FPS
1000.55 FPS
734.94 FPS
1000.79 FPS
Unable to calculate FPS (division by zero)
1000.31 FPS
628.27 FPS
985.74 FPS
Unable to calculate FPS (division by zero)
1002.22 FPS
1000.31